Import libraries

In [1]:
import pprint
import pandas as pd
import numpy as np
import datetime
from bs4 import BeautifulSoup
import requests
import time
import os

In [2]:
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0.3 Safari/605.1.15'}

# Scrape the list of political subReddits

Instruction: https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c

In [3]:
url="https://www.reddit.com/r/redditlists/comments/josdr/list_of_political_subreddits/"

In [4]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup
req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})

webpage = urlopen(req).read()
page_soup = soup(webpage, "html.parser")
title = page_soup.find("title")
print(title)
containers = page_soup.findAll('a',class_="_3t5uN8xUmg0TOwRCOGQEcU")

names=[]
urls=[]
pairs=[]

for container in containers[:174]:
    names.append(container.text)
    urls.append(container['href'])
    pairs.append((container.text,container['href']))
    print(container.text)
    print(container['href'])

subreddit=pd.DataFrame(pairs,columns=['name','url'])


<title>List of political subreddits. : redditlists</title>
World News
http://www.reddit.com/r/worldnews
News
http://www.reddit.com/r/news
World Politics
http://www.reddit.com/r/worldpolitics
World Events
http://www.reddit.com/r/Worldevents
Business
http://reddit.com/r/business
Economics
http://reddit.com/r/Economics
Environment
http://reddit.com/r/environment
Energy
http://www.reddit.com/r/energy
Law
http://reddit.com/r/law/
Education
http://www.reddit.com/r/education/
Government
http://www.reddit.com/r/government/
History
http://www.reddit.com/r/history/
Politics PDF's
http://www.reddit.com/r/PoliticsPDFs/
Wikileaks
http://www.reddit.com/r/WikiLeaks/
SOPA
http://www.reddit.com/r/SOPA/
News Porn
http://www.reddit.com/r/NewsPorn
World News 2
http://www.reddit.com/r/worldnews2/
Anarchist News
http://www.reddit.com/r/AnarchistNews/
Republic of Politics
http://www.reddit.com/r/republicofpolitics
LGBT News
http://www.reddit.com/r/LGBTnews/
Politics 2
http://www.reddit.com/r/politics2/
Econo

In [5]:
subreddit

,name,url
0,World News,http://www.reddit.com/r/worldnews
1,News,http://www.reddit.com/r/news
2,World Politics,http://www.reddit.com/r/worldpolitics
3,World Events,http://www.reddit.com/r/Worldevents
4,Business,http://reddit.com/r/business
...,...,...
169,9/11 Truth,http://www.reddit.com/r/911truth
170,Climate Skeptics,http://www.reddit.com/r/climateskeptics
171,Info Graffiti,http://www.reddit.com/r/infograffiti
172,Conspiracy Hub,http://www.reddit.com/r/conspiracyhub


In [7]:
subreddit.to_csv('Data/subReddit_list.csv')

# Scrape Reddit Post

In [3]:
import requests

# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('fQ-VABpboAY8iQ', 'v6X0ifFGgMRop3g2TlbV8cyX0LXxOQ')

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': 'JoyceOoops',
        'password': '19960703jyr'}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot/0.0.1'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
# Add headers=headers to every request
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)


<Response [200]>

In [4]:
import praw

reddit = praw.Reddit(client_id = 'fQ-VABpboAY8iQ', 
                     client_secret = 'v6X0ifFGgMRop3g2TlbV8cyX0LXxOQ', 
                     user_agent = 'JoyceOoops')

In [11]:
for x in range(len(names))[17:]:
    
    subreddit = reddit.subreddit(urls[x].split('/')[4]) 
    posts = subreddit.hot(limit=1000)

    topics_dict = { "subreddit":[],
                   "title":[],
                   "score":[], 
                   "id":[], 
                   "url":[], 
                   "comms_num": [], 
                   "created": [], 
                   "body":[],
                   "search_type":[]}

    for submission in posts:
        topics_dict["subreddit"].append(names[x])
        topics_dict["id"].append(submission.id)
        topics_dict["title"].append(submission.title)
        topics_dict["score"].append(submission.score)
        topics_dict["url"].append(submission.url)
        topics_dict["comms_num"].append(submission.num_comments)
        topics_dict["created"].append(datetime.datetime.fromtimestamp(submission.created))
    #datetime.datetime.fromtimestamp(submission.created)
        topics_dict["body"].append(submission.selftext)
        topics_dict["search_type"].append("new")

    topics_data = pd.DataFrame(topics_dict)
    topics_data.head()

    topics_data.to_csv("Data/PRAW/PRAW_"+names[x]+".csv")
    print(x,names[x])
    

Forbidden: received 403 HTTP response

In [6]:
dfs = []
docs_location = "Data/PRAW/"

for doc in os.listdir(docs_location):
    print(doc)
    df = pd.read_csv(os.path.join(docs_location + doc))
    dfs.append(df)

PRAW_Business.csv
PRAW_World News.csv
PRAW_Energy.csv
PRAW_World Events.csv
PRAW_World News 2.csv
PRAW_News.csv
PRAW_History.csv
PRAW_Economics.csv
PRAW_Politics PDF's.csv
PRAW_Environment.csv
PRAW.csv
PRAW_Government.csv
PRAW_Law.csv
PRAW_Wikileaks.csv
PRAW_News Porn.csv
PRAW_Education.csv
PRAW_World Politics.csv
PRAW_SOPA.csv


In [12]:
df=pd.concat(dfs, sort=False)
df=df.reset_index(drop=True)
df=df.drop(columns=['Unnamed: 0'])
df.head()

,subreddit,title,score,id,url,comms_num,created,body,search_type
0,Business,Posts regarding politics,81,kurvl4,https://www.reddit.com/r/business/comments/kur...,0,2021-01-11 01:15:49,"Many of you know, we have a strict no-politics...",new
1,Business,USDOT report shows autonomous trucks will only...,271,m8izf6,https://www.traffictechnologytoday.com/news/au...,58,2021-03-19 15:54:04,NaN,new
2,Business,"NFL announces new TV deals, with Amazon gettin...",314,m8ganl,https://www.washingtonpost.com/sports/2021/03/...,67,2021-03-19 13:37:02,NaN,new
3,Business,Ford to hold some F-150s without certain modul...,197,m8j14u,https://www.autonews.com/manufacturing/ford-ho...,39,2021-03-19 15:56:21,NaN,new
4,Business,Lyft says it just had the most riders in a sin...,625,m7zhs6,https://www.cnbc.com/2021/03/18/lyft-says-it-j...,25,2021-03-18 21:09:32,NaN,new


In [111]:
df.to_csv("Data/PRAW_new.csv")

# Name Entity Recognition

In [13]:
df

,subreddit,title,score,id,url,comms_num,created,body,search_type
0,Business,Posts regarding politics,81,kurvl4,https://www.reddit.com/r/business/comments/kur...,0,2021-01-11 01:15:49,"Many of you know, we have a strict no-politics...",new
1,Business,USDOT report shows autonomous trucks will only...,271,m8izf6,https://www.traffictechnologytoday.com/news/au...,58,2021-03-19 15:54:04,NaN,new
2,Business,"NFL announces new TV deals, with Amazon gettin...",314,m8ganl,https://www.washingtonpost.com/sports/2021/03/...,67,2021-03-19 13:37:02,NaN,new
3,Business,Ford to hold some F-150s without certain modul...,197,m8j14u,https://www.autonews.com/manufacturing/ford-ho...,39,2021-03-19 15:56:21,NaN,new
4,Business,Lyft says it just had the most riders in a sin...,625,m7zhs6,https://www.cnbc.com/2021/03/18/lyft-says-it-j...,25,2021-03-18 21:09:32,NaN,new
...,...,...,...,...,...,...,...,...,...
13628,SOPA,Make Congress embarrassed of their support for...,3,sev7u,http://torrentfreak.com/hey-congress-i-visited...,0,2012-04-17 22:30:35,NaN,new
13629,SOPA,Stop Cyber Spying Week Launches Protest,12,se55b,http://www.infozine.com/news/stories/op/storie...,0,2012-04-17 14:16:29,NaN,new
13630,SOPA,"Netflix forms PAC, positions itself to support...",441,rzcrn,http://www.politico.com/news/stories/0412/7492...,113,2012-04-08 17:27:52,NaN,new
13631,SOPA,"Reddit, there's a really simple way that we ca...",4,pls8g,https://www.reddit.com/r/SOPA/comments/pls8g/r...,0,2012-02-12 06:20:24,"I've been running this all day, but I think it...",new


In [14]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [19]:
import spacy

nlp = spacy.load("en_core_web_sm")

entities=[]

for x in df['title']:
    doc = nlp(x)
    for ent in doc.ents:
        entities.append((ent.text,ent.label_))

In [20]:
NER_lists=[]

for r in range(len(df.title)):
    doc = nlp(df.title[r])
    for ent in doc.ents:
        NER_lists.append((ent.text, ent.start_char, ent.end_char, ent.label_,df.title[r],df.id[r],df.subreddit[r]))

NER=pd.DataFrame(NER_lists,columns=['TEXT','START','END','LABEL','Sentence','Article','subreddit'])


In [21]:
NER.to_csv('Data/NER.csv')

In [22]:
df.to_csv('Reddit_SampleData2.csv')

In [17]:
NER=pd.read_csv('Data/NER.csv')

In [18]:
NER

,Unnamed: 0,TEXT,START,END,LABEL,Sentence,Article
0,0,USDOT,0,5,ORG,USDOT report shows autonomous trucks will only...,m8izf6
1,1,NFL,0,3,ORG,"NFL announces new TV deals, with Amazon gettin...",m8ganl
2,2,Amazon,33,39,ORG,"NFL announces new TV deals, with Amazon gettin...",m8ganl
3,3,Thursdays,48,57,DATE,"NFL announces new TV deals, with Amazon gettin...",m8ganl
4,4,17,64,66,CARDINAL,"NFL announces new TV deals, with Amazon gettin...",m8ganl
...,...,...,...,...,...,...,...
23729,23729,a week,31,37,DATE,Several groups Monday launched a week of prote...,se36z
23730,23730,Congress,5,13,ORG,Make Congress embarrassed of their support for...,sev7u
23731,23731,CISPA,47,52,ORG,Make Congress embarrassed of their support for...,sev7u
23732,23732,Iran,53,57,GPE,"Reddit, there's a really simple way that we ca...",pls8g


In [13]:
from collections import Counter

In [15]:
Counter(NER[NER['LABEL']=='PERSON'].subreddit)

Counter({'Business': 32,
         'World News': 211,
         'Energy': 174,
         'World Events': 146,
         'World News 2': 267,
         'News': 75,
         'History': 178,
         'Economics': 135,
         "Politics PDF's": 119,
         'Environment': 121,
         'IWW': 192,
         'Government': 180,
         'Law': 454,
         'Wikileaks': 411,
         'News Porn': 334,
         'Education': 19,
         'World Politics': 155,
         'SOPA': 223})

In [16]:
Counter(NER.subreddit)

Counter({'Business': 234,
         'World News': 1721,
         'Energy': 1776,
         'World Events': 1304,
         'World News 2': 1988,
         'News': 683,
         'History': 1371,
         'Economics': 1047,
         "Politics PDF's": 800,
         'Environment': 1248,
         'IWW': 1243,
         'Government': 1465,
         'Law': 1735,
         'Wikileaks': 1650,
         'News Porn': 3026,
         'Education': 225,
         'World Politics': 740,
         'SOPA': 1478})